In [1]:
import os
import json
import glob
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
import webdataset as wds
from PIL import Image
from io import BytesIO

In [ ]:
training_dir = r"/home/work/llm_data/datasets/food-images/Training"
labels_dir = r"/home/work/llm_data/datasets/food-images/Labels"
meta_file = r"/home/work/llm_data/datasets/food-images/metadata.jsonl"

In [ ]:
img_dirs = list(os.listdir(training_dir))
label_dirs = list(os.listdir(labels_dir))

In [ ]:
all_image_folders = glob.glob("/home/work/llm_data/datasets/Training/*")

In [ ]:
sorted(all_image_folders)

In [ ]:
all_label_folders = glob.glob("/home/work/llm_data/datasets/Labels/*")

In [ ]:
# ll = normalize_folder_name(os.path.basename(sorted(all_label_folders)[103]))

In [ ]:
for i in range(len(ll)):
    print(ll[i], tt[i], ll[i] == tt[i])

In [ ]:
# 한글-영어 음식명 딕셔너리 생성
name_dict = {}
with open('name_dict.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(', ')
        if len(parts) == 3:
            _, korean, english = parts
            name_dict[korean] = english

In [ ]:
def normalize_folder_name(name):
    # 'json' 접미사 제거 및 앞뒤 공백 제거
    return name.replace(' json', '').strip().replace('.json', '')

# 데이터프레임 생성을 위한 리스트
data = []

# Training 폴더의 경로를 기준으로 매칭
for train_dir in all_image_folders:
    text = os.path.basename(train_dir)
    normalized_text = normalize_folder_name(text)
    
    # Labels 폴더에서 매칭되는 경로 찾기
    matching_label_dir = next(
        (label_dir for label_dir in all_label_folders 
         if normalize_folder_name(os.path.basename(label_dir)) == normalized_text),
        None
    )
    
    if matching_label_dir:
        data.append({
            'han_text': text,
            'train_dir': train_dir,
            'label_dir': matching_label_dir
        })
    else:
        print(f'No matching label for {text}, {normalized_text}, {train_dir}')
        # break

# 데이터프레임 생성
df = pd.DataFrame(data)

In [ ]:
df['text'] = df.han_text.map(name_dict)

In [ ]:
def process_files(row):
    results = []
    
    # train_dir에서 jpg 파일 목록 가져오기
    jpg_files = [f for f in os.listdir(row['train_dir']) if f.endswith('.jpg')]
    
    for jpg_file in jpg_files:
        jpg_path = os.path.join(row['train_dir'], jpg_file)
        json_file = os.path.splitext(jpg_file)[0] + '.json'
        json_path = os.path.join(row['label_dir'], json_file)
        
        if os.path.exists(json_path):
            with open(json_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            
            if json_data and isinstance(json_data, list):
                item = json_data[0]  # 첫 번째 항목 사용
                
                result = {
                    'file_name': f"{row['han_text']}/{jpg_file}",
                    # 'image_path': jpg_path,
                    'text': row['text'],
                    'name': item.get('Name', ''),
                    'Cat 1': item.get('Cat 1', ''),
                    'Cat 2': item.get('Cat 2', ''),
                    'Cat 3': item.get('Cat 3', ''),
                    'Cat 4': item.get('Cat 4', '')
                }
                results.append(result)
    
    return results

In [ ]:
# 결과를 저장할 jsonl 파일 경로
output_jsonl = 'metadata.jsonl'

# jsonl 파일에 결과 저장
with open(output_jsonl, 'w', encoding='utf-8') as f:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing and saving files"):
        results = process_files(row)
        for result in results:
            json.dump(result, f, ensure_ascii=False)
            f.write('\n')

# load_dataset('image_folder'): 실패

너무 오래 걸림

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
ds = load_dataset('imagefolder', data_dir='/home/work/llm_data/datasets/food-images/', streaming=True, logger=logging.getLogger())

# WebDataSet 으로 시도

2,373,670 건의 이미지

In [ ]:
def create_webdataset(jsonl_path, image_root, output_path, samples_per_shard=20000):
    writer = wds.TarWriter(output_path)
    sample_count = 0
    shard_count = 0

    with open(jsonl_path, 'r') as f:
        for line in tqdm(f):
            data = json.loads(line)
            image_path = os.path.join(image_root, data['file_name'])
            
            if not os.path.exists(image_path):
                print(f"Warning: Image not found - {image_path}")
                continue

            # 이미지를 바이트로 읽기
            with open(image_path, 'rb') as img_file:
                image_bytes = img_file.read()

            # WebDataset 샘플 생성
            sample = {
                "__key__": f"sample_{sample_count}",
                "jpg": image_bytes,
                "json": json.dumps(data)
            }
            writer.write(sample)

            sample_count += 1

            # 새 샤드 시작
            if sample_count % samples_per_shard == 0:
                writer.close()
                shard_count += 1
                writer = wds.TarWriter(f"{output_path}_{shard_count:05d}.tar")
                # break

    writer.close()
    print(f"Created {shard_count + 1} shards with {sample_count} samples in total.")


In [2]:
jsonl_path = '/home/work/llm_data/datasets/food-images/metadata.jsonl'
image_root = '/home/work/llm_data/datasets/food-images/Training'
output_path = '/home/work/llm_data/datasets/food-images/webdataset/data'

In [ ]:
# create_webdataset(jsonl_path, image_root, output_path)

In [26]:
dataset = load_dataset("webdataset", data_dir='/home/work/llm_data/datasets/food-images/webdataset', streaming=True)

Resolving data files:   0%|          | 0/119 [00:00<?, ?it/s]

In [15]:
next(iter(dataset['train']))

{'__key__': 'sample_1',
 '__url__': '/home/work/llm_data/datasets/food-images/webdataset/data',
 'jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2700x1800>,
 'json': {'Cat 1': '26',
  'Cat 2': '01',
  'Cat 3': '57',
  'Cat 4': 'xx',
  'file_name': '파프리카빨강/A260157XX_02686.jpg',
  'name': 'papeulika,ppalgang',
  'text': 'Red bell pepper'},
 'text': 'Red bell pepper'}

In [12]:
def extract_text(example):
    example['text'] = example['json']['text']
    return example

In [23]:
dataset = dataset.map(extract_text)

In [14]:
print(dataset['train'].column_names)
print(next(iter(dataset['train'])))

None
{'__key__': 'sample_0', '__url__': '/home/work/llm_data/datasets/food-images/webdataset/data', 'jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1748x1309 at 0x7FDED7617760>, 'json': {'Cat 1': '03', 'Cat 2': '02', 'Cat 3': '02', 'Cat 4': 'xx', 'file_name': '볶음쌀국수/B030202XX_31447.jpg', 'name': 'bokk-eumssalgugsu', 'text': 'Stir-fried rice noodles'}, 'text': 'Stir-fried rice noodles'}


In [24]:
dataset['train']

IterableDataset({
    features: Unknown,
    n_shards: 119
})